In [156]:
import pandas as pd
import numpy as np

import plotly.express as px

### Reading file where levesl of BCR::ABL1 is measured : 

In [157]:
pcr  =  pd.read_excel("../mergedPCR_0713.xls")

pcr.columns

Index(['STY1A', 'CTR1N', 'SBJ1N', 'SID1A', 'STYSID1A', 'VISNAM1A', 'VIS1N',
       'QUALIFYV', 'SUBEVE', 'SCHED1C', 'SPCCOL1D', 'SPCCOL1T', 'MUTEUT1D',
       'SPY1A', 'UNTS1', 'MUTTYP1A', 'UNTS2', 'MRKEVL1C', 'SPCCOL1O',
       'MUTEUT1O', 'MUTMPX1C', 'BCRCOP2N', 'ABLCOP3N', 'BCRCOP3N', 'BCRRAW1N',
       'BCRIS1N', 'MUTSTA1C', 'REC1N', 'MUTPCT1N'],
      dtype='object')

Changing column names 

In [158]:
data = pcr[['SID1A', 'VISNAM1A', 'BCRIS1N']]

data.rename(columns = {'SID1A':'patient_id', 'VISNAM1A' : 'month', 'BCRIS1N' : 'BCRABL' }, inplace = True) 

data['month'].value_counts()

/tmp/ipykernel_903061/473407610.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



month
V01- SCREENING      85
V04- END OF M.03    80
V05- END OF M.06    78
V06- END OF M.09    77
V07- END OF M.12    77
V08- END OF M.15    77
V09- END OF M.17    74
V10- END OF M.18    74
V11- END OF M.21    72
V12- EOS            61
UNSCHEDULED          3
Name: count, dtype: int64

Changing the time values

In [159]:
data

,patient_id,month,BCRABL
0,3101_00001,V01- SCREENING,.
1,3101_00001,V04- END OF M.03,0.42
2,3101_00001,V05- END OF M.06,0.4
3,3101_00001,V06- END OF M.09,0.081
4,3101_00001,V07- END OF M.12,0.034
...,...,...,...
754,4810_00013,V04- END OF M.03,1.68
755,4810_00013,V05- END OF M.06,0.36
756,4810_00013,V06- END OF M.09,0.2
757,4810_00013,V07- END OF M.12,0.14


In [160]:
replacements = {
    'V04- END OF M.03'      : '3',
    'V05- END OF M.06'      : '6',
    'V06- END OF M.09'      : '9', 
    'V07- END OF M.12'      : '12',
    'V08- END OF M.15'      : '15',
    'V09- END OF M.17'      : '17', 
    'V10- END OF M.18'      : '18', 
    'V11- END OF M.21'      : '21'
}

data['month'] = data['month'].replace(replacements)

data = data[data.month != 'V01- SCREENING']
data = data[data.month != 'UNSCHEDULED']
data = data[data.month != 'V12- EOS']
data = data[data.month != '21']


/tmp/ipykernel_903061/3234441654.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Find patients we have cytof data from

In [161]:
data

,patient_id,month,BCRABL
1,3101_00001,3,0.42
2,3101_00001,6,0.4
3,3101_00001,9,0.081
4,3101_00001,12,0.034
5,3101_00001,15,0.018
...,...,...,...
754,4810_00013,3,1.68
755,4810_00013,6,0.36
756,4810_00013,9,0.2
757,4810_00013,12,0.14


In [162]:
cytof_patients = pd.read_csv("../patients.csv", header=None, names=['patient_number', 'patient_id', 'batch'])

Removing patients without cytofdata

In [163]:
cytof_patients

,patient_number,patient_id,batch
0,patient_numer,patient_id,batch
1,01,3581_00001,4
2,02,3101_00013,1
3,03,4702_00002,1
4,04,4303_00003,1
5,05,4701_00003,1
6,06,4703_00001,1
7,07,4704_00002,1
8,08,4704_00003,2
9,09,4702_00001,2


In [164]:
patient_ids = list(cytof_patients['patient_id'].unique())
data = data[data['patient_id'].isin(patient_ids)]


## Mapping patient ids to patient number

In [165]:
data = pd.merge(cytof_patients, data, on='patient_id' )

data


,patient_number,patient_id,batch,month,BCRABL
0,01,3581_00001,4,3,1.9
1,01,3581_00001,4,6,0.032
2,01,3581_00001,4,9,0.044
3,01,3581_00001,4,12,0.018
4,01,3581_00001,4,15,0.0038
...,...,...,...,...,...
309,57,4306_00004,6,9,0.025
310,57,4306_00004,6,12,.
311,57,4306_00004,6,15,.
312,57,4306_00004,6,17,0.007


Removing patients that does not haveBCR::ABL measures at 18 months:

### INkluder de som er med til og med måned 3 og måned 6, for da kan du gjøre SL på de også 

In [166]:
unique_patients = data[['patient_id', 'patient_number']].drop_duplicates()
unique_patients

,patient_id,patient_number
0,3581_00001,01
7,3101_00013,02
14,4702_00002,03
21,4303_00003,04
28,4701_00003,05
35,4703_00001,06
42,4704_00002,07
49,4704_00003,08
56,4702_00001,09
63,4702_00004,11


In [167]:
data_with_18_months = data[data['month']  == '18']

patients_18 = data_with_18_months['patient_id'].unique()

data = data[data['patient_id'].isin(patients_18)]

Setting month and response to numerical values 

In [168]:
data['BCRABL'] = pd.to_numeric(data['BCRABL'], errors='coerce')
data['month'] = pd.to_numeric(data['month'], errors='coerce')

/tmp/ipykernel_903061/2553192029.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_903061/2553192029.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [169]:
data.to_csv("responses_all.csv", index=False)

### Plotting the response and the MR-4

In [170]:
import plotly.express as px

unique_batches = data['batch'].unique()
figures = []

# Create a scatter plot for each batch
for batch in unique_batches:
    df_subset = data[data['batch'] == batch]
    df_subset['BCRABL'] = df_subset['BCRABL'].replace(0, 0.001)
    
    # Create the line plot
    fig = px.line(df_subset, x='month', y='BCRABL', color='patient_number', log_y=True,
                  title=f'BCR-ABL% over Time by Patient for Batch {batch}', markers=True)
    
    # Update layout with custom axis titles and set y-axis to logarithmic
    fig.update_layout(
        yaxis_title='BCR-ABL%',
        xaxis_title='Test Time (Months)',
        yaxis_type='log',
        xaxis=dict(
            tickmode='array',
            tickvals=[3, 6, 9, 12, 15, 18],  # Explicitly set ticks for months 6 and 18
            ticktext=['3', '6', '9', '12', '15', '18']  # Custom labels for these ticks
        )
    )

    # Update traces to connect gaps
    fig.update_traces(connectgaps=True)
    
    # Add a horizontal line at y = 0.01 with annotation above the line
    fig.add_hline(y=0.01)
    
    # Add a horizontal line at y = 1 with annotation above the line
    fig.add_hline(y=1, line_dash="dash")

    figures.append(fig)


/tmp/ipykernel_903061/2451909360.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Home/siv32/fiv012/.conda/envs/master/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/tmp/ipykernel_903061/2451909360.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Home/siv32/fiv012/.conda/envs/master/lib/pyth

In [171]:
figures[0].show()

In [172]:
figures[1].show()

In [173]:
figures[2].show()

In [174]:
figures[3].show()

In [175]:
figures[4].show()

In [176]:
figures[5].show()

## Saving the response to plot into dataset

In [177]:
df_response_18 = data[data['month'] == 18]

df_response_18['BCRABL'].isna().sum()

4

In [178]:
df_response_6 = data[data['month'] == 6]

df_response_6
df_response_6 = df_response_6[['patient_number', 'BCRABL']]

df_response_6['CHR'] = df_response_6['BCRABL'].apply(lambda x: 0 if x > 1 else 1)

df_response_6['patient_number'].astype(str)

df_response_6.to_pickle('CHR_6months.pkl')

In [179]:
df_response_18 = data[data['month'] == 18]

df_response_18
df_response_18 = df_response_18[['patient_number', 'BCRABL']]

df_response_18['MR4'] = df_response_18['BCRABL'].apply(lambda x: 0 if x > 0.01 else 1)

df_response_18['patient_number'].astype(str)

df_response_18.to_pickle('MR4_18months.pkl')

In [180]:
df_response_6

,patient_number,BCRABL,CHR
1,01,0.0320,1
8,02,0.0075,1
15,03,0.0760,1
22,04,0.6000,1
29,05,0.6300,1
36,06,3.3000,0
43,07,0.0054,1
50,08,0.1200,1
57,09,0.0350,1
64,11,0.0000,1
